# HW4
- 開始寫作業前，若使用 Colab 請先設定使用 GPU!!
- 資料夾結構
```
📦 HW4
┣ helper_funcs.py
┣ main_hw4.ipynb
┣ model.py
┣ requirements.txt
```

In [2]:
# 0. 安裝套件

!pip install torch==2.6.0 --index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached medmnist-3.0.2-py3-none-any.whl.metadata (14 kB)
  Using cached torchvision-0.22.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using c

In [ ]:
# 1. 載入套件

from tqdm import tqdm # progress bar
# PyTorch 系列
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

from medmnist import PneumoniaMNIST # 本作業資料集

from transformers import AutoModelForImageClassification
from transformers import AutoImageProcessor
from transformers import ViTModel

# Hugging Face PEFT
from peft import LoraConfig, get_peft_model

# Custom modules
from helper_funcs import do_test, load_HF_model
from model import CustomViTClassifier

In [ ]:
# 2. 超參數

BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-4
IMAGE_SIZE = 128
PATCH_SIZE = 16
MODEL_NAME = "google/vit-base-patch16-224" # or google/vit-base-patch16-224-in21k
MODEL_TYPE = "custom"
assert MODEL_TYPE in ["HF", "custom"], "IMPLEMENTATION must be either 'HF' or 'custom'"
SAVE_NAME = f"lora-vit-pneumonia_ep{EPOCHS}"

In [ ]:
tmp = PneumoniaMNIST(split="test", download=True)

In [ ]:
print(tmp)

In [ ]:
# 3. 影像資料前處理準備

image_processor = AutoImageProcessor.from_pretrained(MODEL_NAME, do_rescale=False)
normalize = Normalize(
    mean=[image_processor.image_mean[0]],
    std=[image_processor.image_std[0]],
)
# Data Transform
transform = transforms.Compose(
    [
        transforms.Resize((image_processor.size["height"], image_processor.size["width"])),
        transforms.ToTensor(),
        normalize, # 可以改成你喜歡的
    ]
)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
# 4. Load PneumoniaMNIST
train_raw = PneumoniaMNIST(split="train", download=True, transform=transform)
test_raw = PneumoniaMNIST(split="test", download=True, transform=transform)
train_data, val_data = torch.utils.data.random_split(train_raw, [0.8, 0.2])

100%|██████████| 4.17M/4.17M [00:01<00:00, 2.71MB/s]


In [ ]:
print(type(tmp))
tmp[1][0]

<class 'medmnist.dataset.PneumoniaMNIST'>


In [ ]:
# 5. 創建 DataLoader (TODO1)

train_loader = DataLoader(#  Write your code here)
val_loader = DataLoader(#  Write your code here)
test_loader = DataLoader(# Write your code here)

In [ ]:
# 6. 載入 ViT 模型

if MODEL_TYPE == "HF":
    # Load ViT Model
    model = load_HF_model(MODEL_NAME, num_labels=2)
elif MODEL_TYPE == "custom":
    base_model = ViTModel.from_pretrained(MODEL_NAME)
    model = CustomViTClassifier(base_model, num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 7. 設定 LoRA 參數

lora_config = LoraConfig(
    # task_type=TaskType.IMAGE_CLASSIFICATION, # ViT is a (image) sequence classification model
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"], # https://github.com/huggingface/peft/issues/2128
    bias="none", # 是否包含偏置項
    modules_to_save=["classifier"],
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# 印出可訓練參數的數量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"可訓練參數數量: {trainable_params} ({trainable_params/total_params:.2%})")

In [ ]:
# 8. 訓練前準備

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): CustomViTClassifier(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): ViTPatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0-11): 12 x ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (defaul

In [ ]:
# 9. PyTorch training loop

for epoch in range(EPOCHS):
    pbar = tqdm(train_loader)
    pbar.set_description(f"Training epoch [{epoch+1}/{EPOCHS}]")
    model.train()
    for pixel_values, labels in pbar:
        pixel_values = pixel_values.repeat(1, 3, 1, 1)  # 複製通道 3 次

        # TODO2-1
        # A part of the training script is missing.
        # Write your code here

        if MODEL_TYPE == "HF":
            outputs = model(pixel_values=pixel_values, labels=labels).logits
        elif MODEL_TYPE == "custom":
            outputs = model(pixel_values=pixel_values)

        loss = loss_fn(outputs, labels)
        # TODO2-2
        # A part of the training script is missing.
        # Write your code here
        pbar.set_postfix(loss = loss.item())

    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {loss.item():.4f}')

    do_test(val_loader, model, MODEL_TYPE, loss_fn, device, EPOCHS, epoch)

Training epoch [1/3]:  51%|█████     | 60/118 [00:42<00:41,  1.40it/s, loss=0.295]


KeyboardInterrupt: 

In [ ]:
# 10. test the trained model
do_test(test_loader, model, MODEL_TYPE, loss_fn, device, EPOCHS, epoch, "test")